# Human readable duration format

https://www.codewars.com/kata/52742f58faf5485cae000b9a/train/python

Your task in order to complete this Kata is to write a function which formats a duration, given as a number of seconds, in a human-friendly way.

The function must accept a non-negative integer. If it is zero, it just returns "now". Otherwise, the duration is expressed as a combination of years, days, hours, minutes and seconds.

It is much easier to understand with an example:

* For seconds = 62, your function should return 
    "1 minute and 2 seconds"
* For seconds = 3662, your function should return
    "1 hour, 1 minute and 2 seconds"
For the purpose of this Kata, a year is 365 days and a day is 24 hours.

Note that spaces are important.

Detailed rules

The resulting expression is made of components like 4 seconds, 1 year, etc. In general, a positive integer and one of the valid units of time, separated by a space. The unit of time is used in plural if the integer is greater than 1.

The components are separated by a comma and a space (", "). Except the last component, which is separated by " and ", just like it would be written in English.

A more significant units of time will occur before than a least significant one. Therefore, 1 second and 1 year is not correct, but 1 year and 1 second is.

Different components have different unit of times. So there is not repeated units like in 5 seconds and 1 second.

A component will not appear at all if its value happens to be zero. Hence, 1 minute and 0 seconds is not valid, but it should be just 1 minute.

A unit of time must be used "as much as possible". It means that the function should not return 61 seconds, but 1 minute and 1 second instead. Formally, the duration specified by of a component must not be greater than any valid more significant unit of time.

# Peter's breakdown and walkthrough

Let's take this complicated problem and break it down into smaller problems

> The function must accept a non-negative integer. If it is zero, it just returns "now"

In [5]:
def format_duration(seconds):
    if seconds == 0:
        return "now"

now_res = format_duration(0)
if now_res == "now":
    print("All good")
else:
    print(f"{now_res} is not now, try again")
    print 

All good


Moving on to
> Otherwise, the duration is expressed as a combination of years

Lets just make our function return the number of whole years that the seconds represents

In [19]:
seconds_in_a_year = 60 * 60 * 24 * 365
def format_duration(seconds):
    if seconds == 0:
        return "now"
    number_of_years = seconds // seconds_in_a_year
    return number_of_years

res_1 = format_duration(62)
if res_1 == 0:
    print("All good")
else:
    print(f"{res_1} is not 0 , try again")
    print 

res_2 = format_duration(31536000)
if res_2 == 1:
    print("All good")
else:
    print(f"{res_2} is not 1 , try again")
    print 

All good
All good


In [6]:
60 * 60 * 24 * 365

31536000

In [13]:
62 // 31536000

0

In [18]:
import math
math.floor(62354235235 / 31536000)

1977